In [1]:
using GeoArrays
using DataFrames
using EarthEngine
using MLJ
using Plots

In [2]:
# Initialize the EE API
EE.Initialize()

In [ ]:
# Read a Sentinel-2 image
# Note: pipe doesn't work

# s2 = EE.ImageCollection("COPERNICUS/S2_SR") |>
# □ -> EE.filterBounds(□ -> Point(-123.70, 48.55)) |>
# □ -> EE.filterDate(□, "2021-10-29", "2021-10-31") |>
# □ -> EE.sort(□, "CLOUDY_PIXEL_PERCENTAGE") |>
# EE.first

In [62]:
# Read a Sentinel-2 image
ic = EE.ImageCollection("COPERNICUS/S2_SR")
fb = EE.filterBounds(ic, Point(-123.70, 48.55))
fd = EE.filterDate(fb, "2021-10-29", "2021-10-31")
sr = EE.sort(fd, "CLOUDY_PIXEL_PERCENTAGE")
s2 = EE.first(sr)

EarthEngine.Image(PyObject <ee.image.Image object at 0x000000009A355610>)

In [63]:
print(getInfo(bandNames(s2)))

["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "AOT", "WVP", "SCL", "TCI_R", "TCI_G", "TCI_B", "MSK_CLDPRB", "MSK_SNWPRB", "QA10", "QA20", "QA60"]

In [11]:
s2 = EE.select(s2, ["B2", "B3", "B4", "B8"])

EarthEngine.Image(PyObject <ee.image.Image object at 0x000000009A358880>)

In [58]:
# Properties of band 2
EE.getInfo(s2)["bands"][1]

Dict{Any, Any} with 5 entries:
  "data_type"     => Dict{Any, Any}("max"=>65535, "min"=>0, "precision"=>"int",…
  "dimensions"    => [10980, 10980]
  "crs_transform" => [10, 0, 399960, 0, -10, 5400000]
  "id"            => "B2"
  "crs"           => "EPSG:32610"

In [29]:
s2_clip = EE.clip(s2, EE.BBox(-123.90, 48.35, -123.50, 48.85))

EarthEngine.Image(PyObject <ee.image.Image object at 0x000000009A356FA0>)

In [39]:
s2_info = EE.getInfo(s2_clip)

Dict{Any, Any} with 5 entries:
  "bands"      => Dict{Any, Any}[Dict("crs"=>"EPSG:32610", "data_type"=>Dict{An…
  "properties" => Dict{Any, Any}("REFLECTANCE_CONVERSION_CORRECTION"=>1.01298, …
  "id"         => "COPERNICUS/S2_SR/20211030T192529_20211030T192531_T10UDU"
  "type"       => "Image"
  "version"    => 1635703671627943

In [55]:
# EE.getInfo creates a dictionary with all the image properties
# s2_info["properties"]["SPACECRAFT_NAME"]*" : "*s2_info["id"]
s2_info["bands"][1]

Dict{Any, Any} with 6 entries:
  "crs"           => "EPSG:32610"
  "data_type"     => Dict{Any, Any}("max"=>65535, "min"=>0, "precision"=>"int",…
  "dimensions"    => [3001, 4468]
  "crs_transform" => [10, 0, 399960, 0, -10, 5400000]
  "id"            => "B2"
  "origin"        => [3335, 0]

In [ ]:
s2 = GeoArrays.read("../../sentinel2.tif")

In [ ]:
df = reshape(s2, (size(s2)[1]*size(s2)[2], size(s2)[3])) |>
x -> DataFrame(x, :auto)

In [ ]:
df = coerce(df, Count=>Continuous)

In [ ]:
# K-Means
model_kmeans = @load KMeans pkg=Clustering
pipe_kmeans = @pipeline Standardizer() model_kmeans(k=10)
mach_kmeans = machine(pipe_kmeans, df) |> fit!;

In [ ]:
rep_kmeans = report(mach_kmeans)

In [ ]:
df[!, :clusters] = collect(rep_kmeans.k_means[1]);

In [ ]:
kmeans_raster = reshape(Array(df), (size(s2)[1], size(s2)[2], size(s2)[3]+1)) |>
x -> reverse(x, dims=2) |>
GeoArray

In [ ]:
GeoArrays.bbox!(kmeans_raster, GeoArrays.bbox(s2))
GeoArrays.epsg!(kmeans_raster, 32610)
GeoArrays.write!("../../sentinel2_kmeans.tif", kmeans_raster)

In [ ]:
p1 = plot(s2, band=7)
p2 = plot(kmeans_raster, band=14)
plot(p1, p2, layout=(2,1))